# Creating, saving and loading datasets with CSBMs

In [1]:
import torch

from csbms import MultiClassCSBM

In [2]:
torch.set_printoptions(precision=3)
csbm = MultiClassCSBM(n=8, classes=2, dimensions=4)
data_list = [csbm.data]
for i in range(4):
    csbm.evolve()
    data_list.append(csbm.data)

## Simply by hand

In [3]:
for i in range(len(data_list[-1].x)):
    print(f'node_{i}'.ljust(8), f'| features: {csbm.data.x[i]}'.ljust(60), f'|class: {csbm.data.y[i]}'.ljust(10))

node_0   | features: tensor([-0.256, -0.067,  1.231,  1.374])         |class: 1 
node_1   | features: tensor([-0.045,  0.196,  0.897,  0.614])         |class: 1 
node_2   | features: tensor([-0.537, -0.399,  0.789,  0.633])         |class: 1 
node_3   | features: tensor([-0.130, -0.394,  1.002, -0.165])         |class: 1 
node_4   | features: tensor([ 0.220, -0.155,  0.924,  0.133])         |class: 1 
node_5   | features: tensor([0.337, 0.146, 1.133, 0.192])             |class: 1 
node_6   | features: tensor([ 1.377,  0.744, -0.039,  0.019])         |class: 0 
node_7   | features: tensor([0.144, 0.033, 0.733, 0.771])             |class: 1 
node_8   | features: tensor([ 1.056,  0.809, -0.357, -0.215])         |class: 0 
node_9   | features: tensor([ 0.331,  1.148,  0.207, -0.141])         |class: 0 
node_10  | features: tensor([ 1.049,  0.943, -0.203,  0.367])         |class: 0 
node_11  | features: tensor([ 0.725,  0.515, -0.208, -0.297])         |class: 0 
node_12  | features: tensor(

In [4]:
for i in range(len(data_list[2].x)):
    print(f'node_{i}'.ljust(8), f'| features: {csbm.data.x[i]}'.ljust(60), f'|class: {csbm.data.y[i]}'.ljust(10))

node_0   | features: tensor([-0.256, -0.067,  1.231,  1.374])         |class: 1 
node_1   | features: tensor([-0.045,  0.196,  0.897,  0.614])         |class: 1 
node_2   | features: tensor([-0.537, -0.399,  0.789,  0.633])         |class: 1 
node_3   | features: tensor([-0.130, -0.394,  1.002, -0.165])         |class: 1 
node_4   | features: tensor([ 0.220, -0.155,  0.924,  0.133])         |class: 1 
node_5   | features: tensor([0.337, 0.146, 1.133, 0.192])             |class: 1 
node_6   | features: tensor([ 1.377,  0.744, -0.039,  0.019])         |class: 0 
node_7   | features: tensor([0.144, 0.033, 0.733, 0.771])             |class: 1 
node_8   | features: tensor([ 1.056,  0.809, -0.357, -0.215])         |class: 0 
node_9   | features: tensor([ 0.331,  1.148,  0.207, -0.141])         |class: 0 
node_10  | features: tensor([ 1.049,  0.943, -0.203,  0.367])         |class: 0 
node_11  | features: tensor([ 0.725,  0.515, -0.208, -0.297])         |class: 0 
node_12  | features: tensor(

In [5]:
for i, data in enumerate(data_list):
    torch.save(data, f'./data_object_{i}.pt')

In [6]:
data_objects = []
for i in range(5):
    data = torch.load(f'./data_object_{i}.pt')
    data_objects.append(data)

In [7]:
for i in range(len(data_objects[2].x)):
    print(f'node_{i}'.ljust(8), f'| features: {csbm.data.x[i]}'.ljust(60), f'|class: {csbm.data.y[i]}'.ljust(10))

node_0   | features: tensor([-0.256, -0.067,  1.231,  1.374])         |class: 1 
node_1   | features: tensor([-0.045,  0.196,  0.897,  0.614])         |class: 1 
node_2   | features: tensor([-0.537, -0.399,  0.789,  0.633])         |class: 1 
node_3   | features: tensor([-0.130, -0.394,  1.002, -0.165])         |class: 1 
node_4   | features: tensor([ 0.220, -0.155,  0.924,  0.133])         |class: 1 
node_5   | features: tensor([0.337, 0.146, 1.133, 0.192])             |class: 1 
node_6   | features: tensor([ 1.377,  0.744, -0.039,  0.019])         |class: 0 
node_7   | features: tensor([0.144, 0.033, 0.733, 0.771])             |class: 1 
node_8   | features: tensor([ 1.056,  0.809, -0.357, -0.215])         |class: 0 
node_9   | features: tensor([ 0.331,  1.148,  0.207, -0.141])         |class: 0 
node_10  | features: tensor([ 1.049,  0.943, -0.203,  0.367])         |class: 0 
node_11  | features: tensor([ 0.725,  0.515, -0.208, -0.297])         |class: 0 
node_12  | features: tensor(

## With In-Memory-Dataset-Interface

In [17]:
from torch_geometric.data import InMemoryDataset, download_url

In [22]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None, pre_transform=None, pre_filter=None):
        self.data_list = data_list
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['data']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list = self.data_list
        self.save(data_list, self.processed_file_names[0])

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        if idx >= self.len():
            raise ValueError("Index is out of range.")
        return self.data_list[idx]

In [23]:
csbm_data = MyOwnDataset(root='./csbm/', data_list=data_list)

Processing...
Done!


FileNotFoundError: [Errno 2] No such file or directory: 'csbm\\processed\\data.pt'

In [21]:
csbm_data.get(1)

Data(x=[16, 4], edge_index=[2, 0], y=[16], train_mask=[16], validation_mask=[16], test_mask=[16])